# Ensambles
## La idea es poder lograr algo del siguiente estilo:

pred_1 = modelo_1(df, df_test)

pred_2 = modelo_2(df, df_test)

pred_3 = modelo_3(df, df_test)

pred_4 = modelo_4(df, df_test)

## Con esto se puede evaluar y sacar un promedio entre las predicciones o hacer otros tipos de ensambles

In [1]:
from models import pred_with_rf, pred_with_xgboost, pred_with_rf_ensamb, pred_with_xgboost_ensamb
from models import pred_with_rf_ensamb_2, pred_with_xgboost_ensamb_2
from normalize_data import normailize_df, getNormalizedDataset
from sklearn.model_selection import train_test_split
from utils import target_encoding
import pandas as pd
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('data/train.csv',
        index_col='id',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])

df_test = pd.read_csv('data/test.csv',
        index_col='id',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category'
            },
        parse_dates=['fecha'])


In [33]:
pred_2 = pred_with_xgboost(df, df_test)


Despues de filtrar:  (223475, 20)


/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [34]:
pred_1 = pred_with_rf(df, df_test)

In [ ]:
# utilizando metodo de pesos

In [35]:
final_pred = (0.4*pred_1+0.6*pred_2)

In [37]:
res = pd.DataFrame(final_pred, index=df_test.index, columns=['target'])
display(res.head())
res.to_csv("data/submission-rfxg-weights.csv", header=True)

,target
id,
4941,4.297339e+06
51775,1.004408e+06
115253,2.632642e+06
299321,1.565938e+06
173570,5.894365e+05


# Ensamblando dos modelos por promedio
## Submit
## Score: 573k

In [8]:
final_pred = (pred_1+pred_2)/2

res = pd.DataFrame(final_pred, index=df_test.index, columns=['target'])
display(res.head())
res.to_csv("data/submission-rfxg.csv", header=True)

,target
id,
4941,4.706227e+06
51775,9.876805e+05
115253,2.586411e+06
299321,1.533391e+06
173570,5.825126e+05


# Ensamblando dos modelos con un tercero (stacking)

In [3]:
X = df.drop('precio', axis=1)
y = df['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

df_xy = X_train.join(y_train)


In [4]:
pred_1, xgb, te_xg = pred_with_xgboost_ensamb(df_xy, X_test)

/home/ggparente95/Documentos/Facu/Datos/TP2/tp2-zonajobs/normalize_data.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idsNullMetrosTotales]['metrostotales'] =  df[idsNullMetrosTotales]['metroscubiertos']
/home/ggparente95/Documentos/Facu/Datos/TP2/tp2-zonajobs/normalize_data.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idsNullMetrosCubiertos]['metroscubiertos'] =  df[idsNullMetrosCubiertos]['metrostotales']



Despues de filtrar:  (156452, 20)


/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [5]:
pred_2, rf, te_rf  = pred_with_rf_ensamb(df_xy, X_test)

In [10]:

df_p1 = pd.DataFrame(pred_1, index=X_test.index, columns=['RF'])
df_p2 = pd.DataFrame(pred_2, index=X_test.index, columns=['XGB'])
df_preds = df_p1.join(df_p2)
df_preds = df_preds.join(y_test)

## RF ### XGB ## PRECIO
xg_reg = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.8, learning_rate = 0.1, max_depth = 10, reg_alpha = 1.2, n_estimators = 500, reg_lambda = 1.4, subsample=0.8)

xg_reg.fit(df_preds.drop('precio', axis=1), df_preds['precio'])



/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=1.2, reg_lambda=1.4, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

## Los 3 modelos ya estan entrenados. Arrranca el df test final.

In [23]:
# Predecir lo de test
df_test_norm = getNormalizedDataset(df_test, 'test')
X_test, target_enc = target_encoding(df_test_norm, 'test', te_rf)

X_test.ciudad.fillna(X_test.ciudad.value_counts().idxmax(), inplace=True)

pred_1_t = rf.predict(X_test)


df_test_norm_2 = normailize_df(df_test, False)
#df_test_norm_2.drop('precio_mt2', axis=1, inplace=True)
X_test, target_enc = target_encoding(df_test_norm_2, 'test', te_xg)

pred_2_t = xgb.predict(X_test)


/home/ggparente95/Documentos/Facu/Datos/TP2/tp2-zonajobs/normalize_data.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idsNullMetrosTotales]['metrostotales'] =  df[idsNullMetrosTotales]['metroscubiertos']
/home/ggparente95/Documentos/Facu/Datos/TP2/tp2-zonajobs/normalize_data.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idsNullMetrosCubiertos]['metroscubiertos'] =  df[idsNullMetrosCubiertos]['metrostotales']


In [20]:
X_test.dtypes

tipodepropiedad               float64
ciudad                        float64
provincia                     float64
antiguedad                      int64
habitaciones                    int64
garages                         int64
banos                           int64
metroscubiertos                 int64
metrostotales                   int64
gimnasio                        int64
usosmultiples                   int64
piscina                         int64
escuelascercanas                int64
centroscomercialescercanos      int64
extras                        float64
dia                             int64
mes                             int64
anio                            int64
dtype: object

In [26]:
df_rf = pd.DataFrame(pred_1_t, index=df_test.index, columns=['RF'])
df_xg = pd.DataFrame(pred_2_t, index=df_test.index, columns=['XGB'])
df_join = df_rf.join(df_xg)


In [27]:
final_pred = xg_reg.predict(df_join)

# SUBMIT ENSAMBLE USANDO RF | XGB | XGB

In [30]:
res = pd.DataFrame(final_pred, index=df_test.index, columns=['target'])
display(res.head())
res.to_csv("data/submission-stacking-rfxgxg.csv", header=True)

,target
id,
4941,5.387554e+06
51775,9.323980e+05
115253,2.804422e+06
299321,1.544294e+06
173570,5.393741e+05


# SUBMIT USANDO PESOS, PERO CON FEATURE SELECTION

In [3]:
pred_1, rf, t_e = pred_with_rf_ensamb_2(df, df_test)

In [4]:
pred_2, xgb, t_e2 = pred_with_xgboost_ensamb_2(df, df_test)

/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ggparente95/.local/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [10]:
final_pred = (0.5*pred_1+0.5*pred_2)

In [11]:
final_pred

array([6767323.72698498,  918932.35496411, 2175690.53629692, ...,
        764283.71201956, 1339996.57813488, 2213515.37699908])

In [13]:
res = pd.DataFrame(final_pred, index=df_test.index, columns=['target'])
display(res.head())
res.to_csv("data/submission-fs-w-rf-xgb.csv", header=True)

,target
id,
4941,6.767324e+06
51775,9.189324e+05
115253,2.175691e+06
299321,1.369797e+06
173570,5.385316e+05
